In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/ladans/Learning systems


In [3]:
dir_base = os.path.join(os.getcwd(),'hex_game_results_13.csv')
print(dir_base)

/home/ladans/Learning systems/hex_game_results_13.csv


In [4]:
df = pd.read_csv(dir_base)
df = df.iloc[:10000]
df['Moves'] = df['Moves'].apply(lambda x: list(map(int, x.split())))
df['Moves']

0       [85, 99, 108, 134, 47, 12, 110, 76, 142, 165, ...
1       [125, 28, 20, 24, 17, 65, 116, 36, 159, 12, 15...
2       [94, 158, 25, 2, 87, 144, 18, 5, 44, 79, 154, ...
3       [150, 62, 34, 160, 81, 123, 48, 39, 71, 125, 1...
4       [87, 16, 3, 64, 68, 21, 134, 76, 153, 91, 26, ...
                              ...                        
9995    [102, 104, 113, 120, 9, 70, 95, 152, 35, 45, 1...
9996    [14, 57, 130, 58, 19, 96, 120, 46, 13, 145, 34...
9997    [56, 58, 122, 162, 96, 64, 74, 102, 165, 70, 1...
9998    [4, 9, 126, 118, 10, 80, 84, 166, 134, 29, 153...
9999    [167, 34, 111, 47, 24, 84, 145, 117, 103, 17, ...
Name: Moves, Length: 10000, dtype: object

In [5]:
first_cell=df['Moves'].iloc[0]
print(len(first_cell))

164


In [ ]:
def move_to_coordinates(move, board_size):
    row = move // board_size
    col = move % board_size
    return row, col

def create_hex_graph(board_dim, moves):
    G = nx.Graph()
    
    # Add nodes for each hexagonal position on the board
    for i in range(board_dim):
        for j in range(board_dim):
            G.add_node((i, j), player=None)  # Initially, no player has occupied the position
    
    # Add edges between neighboring positions
    for i in range(board_dim):
        for j in range(board_dim):
            neighbors = [
                (i - 1, j), (i + 1, j),          # Vertical neighbors
                (i, j - 1), (i, j + 1),          # Horizontal neighbors
                (i - 1, j + 1), (i + 1, j - 1)   # Diagonal neighbors
            ]
            for neighbor in neighbors:
                if 0 <= neighbor[0] < board_dim and 0 <= neighbor[1] < board_dim:
                    G.add_edge((i, j), neighbor)
    
    # Assign player moves to nodes
    for i, move in enumerate(moves):
        player = 1 if i % 2 == 0 else 2  # Alternate between Player 1 and Player 2
        row, col = move_to_coordinates(move, board_dim)
        if 0 <= row < board_dim and 0 <= col < board_dim:
            G.nodes[(row, col)]['player'] = player
        else:
            print(f"Invalid move: {move} exceeds board size")
    
    return G

def visualize_hex_graph(G):
    pos = {(i, j): (j, -i) for i, j in G.nodes()}  # Arrange nodes in hexagonal layout
    labels = {node: G.nodes[node]['player'] for node in G.nodes()}
    
    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, labels=labels, with_labels=True, node_size=700, node_color='lightgreen')
    plt.show()

#########################################################################################################
##################################### Process and store all games in the dataset########################
#########################################################################################################

In [ ]:
board_dim = 13
game_graphs = []
winners = []


for index, row in df.iterrows():
    game_moves = row['Moves']
    winner = row['Winner']

    game_graph = create_hex_graph(board_dim, game_moves)
    adjacency_matrix = nx.to_numpy_array(game_graph)    # Convert the graph to an adjacency matrix
    game_graphs.append(adjacency_matrix)
    winners.append(winner)

    if index == 0:    # I have stored the first game graph separately to show the graph
        first_game_graph = game_graph


visualize_hex_graph(first_game_graph)# Visualize just the first game graph